<div align="center">
    <div><img src="../assets/redis_logo.svg" style="width: 130px"> </div>
    <div style="display: inline-block; text-align: center; margin-bottom: 10px;">
        <span style="font-size: 36px;"><b>Basic RAG with LangChain</b></span>
        <br />
    </div>
    <br />
</div>

## Environment Setup

In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import os
import warnings
import dotenv
# load env vars from .env file
dotenv.load_dotenv()

warnings.filterwarnings('ignore')
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
sys.path.insert(0, f'{parent_directory}/helpers')
os.environ["ROOT_DIR"] = parent_directory
REDIS_URL = os.getenv("REDIS_URL")

print("========== ENVIRONMENT VARIABLES ==========")
print(f"Current Directory={dir_path}")
print(f"Parent Directory={parent_directory}")
print(f"System path={sys.path}")
print("---------------------------------")
print(f'LLM Engine: {os.getenv("LOCAL_LLM_ENGINE")}')
print(f'LOCAL_VLLM_MODEL: {os.getenv("LOCAL_VLLM_MODEL")}')
print(f'LOCAL_OLLAMA_MODEL: {os.getenv("LOCAL_OLLAMA_MODEL")}')
print(f'VLLM_URL: {os.getenv("VLLM_URL")}')
print("---------------------------------")
print(f"NLTK_DATA={os.getenv('NLTK_DATA')}")

/Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop/1_getting_started
/Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop
['/Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop/helpers', '/Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']


### Install Python Dependencies

In [3]:
%pip install -r $ROOT_DIR/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
from utils import *

### Configure your Redis Stack


In [5]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://{REDIS_HOST}:{REDIS_PORT}"

### SentenceTransformerEmbeddings Models Cache folder
We are using `SentenceTransformerEmbeddings` in this demo and here we specify the cache folder. If you already downloaded the models in a local file system, set this folder here, otherwise the library tries to download the models in this folder if not available locally.

In particular, these models will be downloaded if not present in the cache folder:

models/models--sentence-transformers--all-MiniLM-L6-v2

models/models--sentence-transformers--all-mpnet-base-v2


In [6]:
#setting the local downloaded sentence transformer models f
os.environ["TRANSFORMERS_CACHE"] = f"{parent_directory}/models"

## RAG with LangChain

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = f"{parent_directory}/resources/10K"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['/Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop/resources/10K/nke-10k-2023.pdf']


In [8]:
# pick out the Nike doc for this exercise
doc = [doc for doc in docs if "nke" in doc][0]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    

loader = UnstructuredFileLoader(
    doc, mode="single", strategy="fast"
)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Done preprocessing. Created 180 chunks of the original pdf /Users/rouzbeh.farahmand/PycharmProjects/boa-financial-rag-workshop/resources/10K/nke-10k-2023.pdf


In [9]:
print(f"created {len(chunks)} chunks ")

created 180 chunks 


### Initialize Embeddings Engine
Here we will use LangChain's built in embedding engine so that it will work seemlessly with the LangChain VectorStore classes.

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings 
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", cache_folder=os.getenv("TRANSFORMERS_CACHE", f"{parent_directory}/models"))

## Vector Search with LangChain
### Create Redis vector store instance

We also need to create a schema for the vector index so we can take advantage of the metadata along with the vectors.


**Important Note-1**: If you use your own embedding model with different dimensions, make sure to modify the `dims` in the schema below accordingly.

**Important Note-2**: LangChain does not support JSON data types yet. Only supports HASH for now. This update should be coming soon.

In [11]:
from langchain_community.vectorstores import Redis

index_name = 'basiclangchain'
vector_schema = {
        "name": "content_vector",
        "algorithm": "FLAT",
        "dims": 384,
        "distance_metric": "COSINE",
        "datatype": "FLOAT32",
    }
index_schema = {
    "vector": [vector_schema],
    "text": [{"name": "content"}],
    "content_vector_key": "content_vector"    # name of the vector field in langchain
}

print(f"loading {len(chunks)} chucks to REDIS_URL={REDIS_URL}")
# construct the vector store class from texts and metadata
rds = Redis.from_documents(
    documents=chunks,
    embedding=embeddings,
    vector_schema=vector_schema,
    redis_url=REDIS_URL,
    index_name = index_name
)

loading 180 chucks to REDIS_URL=redis://localhost:6379


In [12]:
# access underlying redis client to see how many docs have been stores
rds.client.dbsize()

3445

### Query the database
Now we can use the LangChain vector store class to perform similarity search operations on Redis

In [13]:
from langchain.vectorstores.redis import RedisText

In [14]:
# basic "top 4" vector search on a given query
rds.similarity_search_with_score(query="Profit margins", k=4)

[(Document(page_content="(Dollars in millions, except per share data)\n\nRevenues Cost of sales\n\nGross profit Gross margin\n\nDemand creation expense Operating overhead expense\n\nTotal selling and administrative expense % of revenues\n\nInterest expense (income), net\n\nOther (income) expense, net Income before income taxes\n\nIncome tax expense Effective tax rate\n\nNET INCOME Diluted earnings per common share\n\n$\n\n$ $\n\nFISCAL 2023\n\n51,217 28,925\n\n22,292\n\n43.5 %\n\n4,060 12,317\n\n16,377\n\n32.0 % (6)\n\n(280) 6,201\n\n1,131\n\n18.2 %\n\n5,070 3.23\n\n$\n\n$ $\n\nFISCAL 2022\n\n46,710 25,231\n\n21,479\n\n46.0 %\n\n3,850 10,954\n\n14,804\n\n31.7 % 205\n\n(181) 6,651\n\n605 9.1 %\n\n6,046 3.75\n\n% CHANGE\n\n10 % $ 15 %\n\n4 %\n\n5 % 12 %\n\n11 %\n\n—\n\n— -7 %\n\n87 %\n\n16 % $ -14 % $\n\nFISCAL 2021\n\n% CHANGE\n\n44,538 24,576\n\n5 % 3 %\n\n19,962\n\n8 %\n\n44.8 %\n\n3,114 9,911\n\n24 % 11 %\n\n13,025\n\n14 %\n\n29.2 % 262\n\n—\n\n14 6,661\n\n— 0 %\n\n934 14.0 %\n\n35 %

In [15]:
# vector search with metadata filtering
f = RedisText("content") % "profit"
rds.similarity_search_with_score(query="Profit margins", k=4, filter=f)

[(Document(page_content="(Dollars in millions, except per share data)\n\nRevenues Cost of sales\n\nGross profit Gross margin\n\nDemand creation expense Operating overhead expense\n\nTotal selling and administrative expense % of revenues\n\nInterest expense (income), net\n\nOther (income) expense, net Income before income taxes\n\nIncome tax expense Effective tax rate\n\nNET INCOME Diluted earnings per common share\n\n$\n\n$ $\n\nFISCAL 2023\n\n51,217 28,925\n\n22,292\n\n43.5 %\n\n4,060 12,317\n\n16,377\n\n32.0 % (6)\n\n(280) 6,201\n\n1,131\n\n18.2 %\n\n5,070 3.23\n\n$\n\n$ $\n\nFISCAL 2022\n\n46,710 25,231\n\n21,479\n\n46.0 %\n\n3,850 10,954\n\n14,804\n\n31.7 % 205\n\n(181) 6,651\n\n605 9.1 %\n\n6,046 3.75\n\n% CHANGE\n\n10 % $ 15 %\n\n4 %\n\n5 % 12 %\n\n11 %\n\n—\n\n— -7 %\n\n87 %\n\n16 % $ -14 % $\n\nFISCAL 2021\n\n% CHANGE\n\n44,538 24,576\n\n5 % 3 %\n\n19,962\n\n8 %\n\n44.8 %\n\n3,114 9,911\n\n24 % 11 %\n\n13,025\n\n14 %\n\n29.2 % 262\n\n—\n\n14 6,661\n\n— 0 %\n\n934 14.0 %\n\n35 %

In [16]:
# vector search with combinations of metadata filtering
f = (RedisText("content") % "profit") | (RedisText("content") % "revenue")
rds.similarity_search_with_score(query="Nike company revenue", k=4, filter=f)

[(Document(page_content='4,780 (508)\n\n7 % -80 %\n\nTOTAL NIKE BRAND WHOLESALE EQUIVALENT REVENUES\n\n$\n\n40,127 $\n\n36,151\n\n11 %\n\n18 % $\n\n35,770\n\n1 %\n\n(1)\n\nThe percent change excluding currency changes and the presentation of wholesale equivalent revenues represent non-GAAP financial measures. For further information, see "Use of Non-GAAP Financial Measures".\n\n(2) Global Brand Divisions revenues include NIKE Brand licensing and other miscellaneous revenues that are not part of a geographic operating segment.\n\n(3) Corporate revenues primarily consist of foreign currency hedge gains and losses related to revenues generated by entities within the NIKE Brand geographic operating segments and Converse, but\n\nmanaged through our central foreign exchange risk management program.\n\n(4)\n\nAs a result of the Consumer Direct Acceleration strategy, announced in fiscal 2021, the Company is now organized around a consumer construct of Men\'s, Women\'s and Kids\'. Beginning in 

In [17]:
# filter results to a certain distance threshold
rds.similarity_search_with_score(query="Nike company revenue", k=4, distance_threshold=0.5)

[(Document(page_content='As discussed in Note 15 — Operating Segments and Related Information in the accompanying Notes to the Consolidated Financial Statements, our operating segments are evidence of the structure of the Company\'s internal organization. The NIKE Brand segments are defined by geographic regions for operations participating in NIKE Brand sales activity.\n\nThe breakdown of Revenues is as follows:\n\n(Dollars in millions)\n\nFISCAL 2023 FISCAL 2022\n\n% CHANGE\n\n% CHANGE EXCLUDING CURRENCY (1) CHANGES FISCAL 2021\n\n% CHANGE\n\nNorth America Europe, Middle East & Africa Greater China\n\n$\n\n21,608 $ 13,418 7,248\n\n18,353 12,479 7,547\n\n18 % 8 % -4 %\n\n18 % $ 21 % 4 %\n\n17,179 11,456 8,290\n\n7 % 9 % -9 %\n\nAsia Pacific & Latin America Global Brand Divisions\n\n(3)\n\n(2)\n\n6,431 58\n\n5,955 102\n\n8 % -43 %\n\n17 % -43 %\n\n5,343 25\n\n11 % 308 %\n\nTOTAL NIKE BRAND Converse\n\n$\n\n48,763 $ 2,427\n\n44,436 2,346\n\n10 % 3 %\n\n16 % $ 8 %\n\n42,293 2,205\n\n5 % 

## RAG with HuggingFace LLM
LangChain makes it easy to now take this vector store and build retireval augmented generation (RAG) applications over your data.

### Initialize a local llama LLM
Alternatively, if you like to connect to a local Ollama LLM, you can use below LLM. If you have a local OpenAI-compatible server running via vLLM or Ollama, add your LLM here.

In [18]:
llm = get_llm( 
        local_llm_engine=os.getenv("LOCAL_LLM_ENGINE"),
        vllm_url=os.getenv("VLLM_URL"),
        vllm_model=os.getenv("LOCAL_VLLM_MODEL"),
        ollama_model=os.getenv("LOCAL_OLLAMA_MODEL"),
        temperature=0)

### Setup prompt
PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [19]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate

    # Define our prompt
    prompt_template = """Use the following pieces of context from financial 10k filings data to answer the user question at the end. Only use the result from tools and evidence provided to you. If you don't know the answer, say that you don't know, don't try to make up an answer. Provide the source of the document that you used to get the answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]
    Source: [source document here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the Langchain brings all the components together in a form of a simple RAG application with the financial PDF document.

In [20]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=rds.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.8}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()},
    verbose=True
)

### Finally - let's ask questions!



In [21]:
query = "What was Nike's revenue last year compared to this year??"
res=qa(query)
res['result']

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


"Based on the provided data, Nike's revenue for the previous year (FISCAL 2022) is:\n\n* TOTAL NIKE, INC. REVENUES: $51,217\n\nAnd for this year (FISCAL 2023), it is:\n\n* TOTAL NIKE, INC. REVENUES: $46,710\n\nSo, there was a decrease of approximately:\n\n* 10% ($4,507)\n\ncompared to the previous year's revenue."

In [22]:
query = "How many products does Nike offer? What is the industry that Nike is part of?"
res=qa(query)
res['result']



> Entering new RetrievalQA chain...

> Finished chain.


'Question: How many products does Nike offer? What is the industry that Nike is part of?\n\nAnswer: According to the provided context, Nike offers a variety of products, including:\n\n* Athletic footwear products (Men\'s, Women\'s, and Jordan Brand)\n* Apparel products\n* Equipment, accessories, and services\n\nAs for the industry that Nike is part of, it is stated that:\n\n"NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this \'Annual Report\'), the terms \'we\', \'us\', \'our\', \'NIKE\' and the \'Company\' refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise."\n\nBased on this information, it can be inferred that Nike is part of the athletic apparel and footwear industry.\n\nNote: The provided text does not explicitly state the number of products offered by Nike.'

In [23]:
query = "Is Nike an ethical company?"
res=qa(query)
res['result']



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know.\n\nThe provided context is the 10-K filing of NIKE, Inc., which primarily discusses the company's business activities, products, and operations. It does not provide any information or insights about the company's ethics or moral character. To determine whether Nike is an ethical company, one would need to examine external reports, ratings, and reviews from reputable sources such as Corporate Social Responsibility (CSR) organizations, NGOs, or industry watchdogs.\n\nSome possible places to look for answers might include:\n\n1. CSR reports: Companies like Dow Jones Sustainability Indexes, Sustainability Accounting Standards Board (SASB), or the Global Reporting Initiative (GRI) provide ratings and assessments of companies' sustainability and social responsibility practices.\n2. Industry reports: Reports from industry associations, trade publications, or research firms may offer insights into a company's ethics and business practices.\n3. NGO/NGI reviews: Non-governmental o

In [24]:
query = "How many employees work at Nike???"
res=qa(query)
res['result']



> Entering new RetrievalQA chain...

> Finished chain.


"The provided financial data does not include information about the number of employees working at Nike. The data appears to be related to revenues, sales-related reserves, and operating segments, but it does not provide details on human resources or employee counts.\n\nTo find the answer to this question, you may want to look for a different type of report, such as an Annual Report (Form 10-K) or a Quarterly Report (Form 10-Q), which may include information about Nike's workforce. Alternatively, you can search for publicly available reports or news articles that may provide details on the company's employee count."

## Cleanup

Cleanup the index and data.

In [25]:
#rds.drop_index(index_name=index_name, redis_url=REDIS_URL, delete_documents=True)